In [23]:
import pandas as pd
import numpy as np
import logging

# Load the data file<
file_path = '/home/student/Cloud/Owncloud/SyncVM/CIP/hslu-cip/Group8__ImholzA_AntonB_GonzalezC/Imholz_Chris_studentA/Data/Imholz_Chris_studentA_stage1.csv'
#file_path = '/home/student/Cloud/Owncloud/SyncVM/CIP/hslu-cip/data/raw_data_stage1/Imholz_Chris_studentA_stage1.csv'
data = pd.read_csv(file_path, delimiter=';')


# Display the first few rows of the dataframe and the general information
data.head()
data.info()

FileNotFoundError: [Errno 2] No such file or directory: '/home/student/Cloud/Owncloud/SyncVM/CIP/hslu-cip/Group8__ImholzA_AntonB_GonzalezC/Imholz_Chris_studentA/Data/Imholz_Chris_studentA_stage1.csv'

In [ ]:
data

In [ ]:
# Check for columns that are completely empty and drop them
data_not_empty_columns = data.dropna(how='all', axis=1)

# Show the columns removed (if any)
data_not_empty_columns.columns.difference(data.columns)
data=data_not_empty_columns

# Updated DataFrame info
data.info()


In [ ]:
# Count rows where all elements are NaN
empty_rows_count = data.isna().all(axis=1).sum()
print("Number of completely empty rows:", empty_rows_count)

# Check for rows that are completely empty and drop them
data = data.dropna(how='all', axis=0)
data


In [ ]:
# Helper function to convert Market Cap to numeric
# Beispiel-Wechselkurse zu USD: Google 18. Apr., 13:08 UTC
exchange_rates = {
'EUR': 1.07,
'GBP': 1.25,
'NOK': 0.091,
'USD': 1
}

def expand_number(value):
    if pd.isna(value):
        return np.nan
    value = value.strip().upper()
    if value.endswith('B'):
        value =  float(value[:-1]) * 1e9  # Convert billions to a numeric value
    elif value.endswith('M'):
        value =  float(value[:-1]) * 1e6  # Convert millions to a numeric value
    elif value.endswith('K'):
        value =  float(value[:-1]) * 1e3  # Convert thousands to a numeric value
    else:
        value = float(value)  # Handle numeric-only strings as is
    return round(value,3)

def convert_to_usd(currency, amount):
    if pd.isna(amount):
        return np.nan
    return amount * exchange_rates[currency]

def process_market_cap(row):
    # Erweitere den Betrag zu einer vollständigen Zahl
    full_amount = expand_number(row['Market Cap'])
    # Wandle die Währung in USD um
    amount_in_usd = convert_to_usd(row['Currency'], full_amount)
    return amount_in_usd

# Lade den DataFrame, hier durch eine CSV-Datei ersetzt
# data = pd.read_csv('path_to_your_csv.csv')

# Annahme: 'data' ist ein DataFrame mit den Spalten 'Currency' und 'Market Cap'
# Anwenden der Funktion
data['Market Cap in USD'] = data.apply(process_market_cap, axis=1)
data = data.drop('Market Cap',axis='columns')

In [ ]:
def parse_and_convert_tons_currency(value):
    # Prüft auf NaN Werte
    if pd.isna(value) or value == '':
        return None
    
    parts = value.split(' ')
    
    if len(parts) < 5:
        return None

    # Extrahiert den Betrag, die Einheit und die Währung
    if parts[0].replace(',', '') == '':
        return None
    else:
        amount = float(parts[0].replace(',', ''))
    
    unit = parts[2]
    currency = parts[4].strip('()')

    # Konvertiert Tausend in Millionen, falls nötig
    if unit == 'Thousand':
        amount /= 1000

    # Wandelt den Betrag in USD um, falls er nicht bereits in USD ist
    if currency != 'USD':
        amount *= round(float(exchange_rates.get(currency,1)),3)

    return amount

# Anwendung der Funktion
data['Carbon footprint (total GHG emissions / enterprise value)' + '_in_USD'] = data['Carbon footprint (total GHG emissions / enterprise value)'].apply(parse_and_convert_tons_currency)
data = data.drop('Carbon footprint (total GHG emissions / enterprise value)',axis='columns')
data

In [ ]:
# Helper function to convert percentage strings to floats
def convert_percentage(value):
    if pd.isna(value) or value.strip('% ') == '':
        return np.nan
    return round(float(value.strip('% ')) / 100,3)

def unit_strip(value):
    if pd.isna(value) or value.strip('unit') == '':
        return np.nan
    elif value.replace('unit', '') == " ":
        return np.nan
    return float(value.replace('unit', ''))

import re
def convert_to_mwh(value):
    conversion_factors = {
    'GWh': 1000,
    'terajoule': 277.778,
    'gigajoule': 0.277778,
    'MWh': 1
    }
    match = re.search(r'([\d,]+(?:\.\d+)?)\s*(GWh|MWh|terajoule|gigajoule)', str(value).replace(',', ''))
    if match:
        # Convert string value to float
        value = float(match.group(1).replace(',', ''))
        unit = match.group(2)
        return round(value * conversion_factors[unit],3) # Convert the value to MWh

# Apply conversions
data['Share of women in total workforce in %'] = data['Share of women in total workforce'].apply(convert_percentage)
data = data.drop("Share of women in total workforce",axis='columns')
data['Share of women in management bodies in %'] = data['Share of women in management bodies'].apply(convert_percentage)
data = data.drop('Share of women in management bodies',axis='columns')
data['Gender pay gap in %'] = data['Gender pay gap'].apply(convert_percentage)
data = data.drop('Gender pay gap',axis='columns')
data['Rate of employees with disabilities in %'] = data['Rate of employees with disabilities'].apply(convert_percentage)
data = data.drop('Rate of employees with disabilities',axis='columns')
data['Board gender diversity (female board members / total board members) in %'] = data['Board gender diversity (female board members / total board members)'].apply(convert_percentage)
data = data.drop('Board gender diversity (female board members / total board members)',axis='columns')
data['Professional equality index'] = data['Professional equality index'].apply(unit_strip)
data['Number of female board members'] = data['Number of female board members'].apply(unit_strip)
data['Number of board members'] = data['Number of board members'].apply(unit_strip)
data['Average training hours per employee'] = data['Average training hours per employee'].apply(unit_strip)
data['Total energy consumption in mwh'] = data['Total energy consumption'].apply(convert_to_mwh)
data = data.drop('Total energy consumption', axis='columns')
data['Rate of resignation in %'] = data['Rate of resignation'].apply(convert_percentage)
data = data.drop('Rate of resignation', axis='columns')
data

# Transformed data
data.head()
data.info()

In [ ]:
data

In [ ]:
# Calculate the percentage of missing values per column
missing_data = data.isnull().mean() * 100.00

# Display the percentage of missing data per column
missing_data_sorted = missing_data.sort_values(ascending=False)
missing_data_sorted


In [ ]:
# Finden der Spalten, bei denen der Prozentsatz der fehlenden Daten größer als 80% ist
columns_to_drop = missing_data[missing_data > 80].index
# Löschen dieser Spalten aus dem DataFrame
data.drop(columns=columns_to_drop, inplace=True)
data

In [ ]:
columns_to_drop

In [ ]:
data.columns

In [ ]:
#Save DataFrame to a CSV file with semicolon as the delimiter with the required _stage2.csv ending form project
data.to_csv('/home/student/Cloud/Owncloud/SyncVM/CIP/hslu-cip/Group8__ImholzA_AntonB_GonzalezC/Imholz_Chris_studentA/Data/Imholz_Chris_studentA_stage3.csv', sep=';', index=False)
# data.to_csv('/home/student/Cloud/Owncloud/SyncVM/CIP/hslu-cip/data/clean_data_stage3/Imholz_Chris_studentA_stage3.csv', sep=';', index=False)

In [ ]:
# # Tabel erstellen 
# CREATE TABLE tbl_financial_euronext_data (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     name VARCHAR(255),
#     currency VARCHAR(255),
#     market_cap_in_USD VARCHAR(255),
#     carbon_footprint_total_GHG_emissions_per_enterprise_value VARCHAR(255),
#     security_type VARCHAR(255),
#     security_type_subtype VARCHAR(255),
#     market VARCHAR(255),
#     isin VARCHAR(20),
#     industry VARCHAR(255),
#     super_sector VARCHAR(255),
#     sector VARCHAR(255),
#     subsector VARCHAR(255),
#     average_training_hours_per_employee VARCHAR(255),
#     number_of_female_board_members VARCHAR(255),
#     number_of_board_members VARCHAR(255),
#     share_of_woman_in_total_workforce_in_percent VARCHAR(255),
#     share_of_woman_in_management_bodies_in_percent VARCHAR(255),
#     gender_pay_gap_in_percent VARCHAR(255),
#     rate_of_employees_with_disabilities_in_percent VARCHAR(255),
#     board_gender_diversity_in_percent VARCHAR(255),
#     total_energy_consumption_in_mwh VARCHAR(255),
#     rate_of_resignation_in_percent VARCHAR(255),
# );

In [ ]:
import mariadb
import sys
import pandas as pd


data = data.where(pd.notnull(data), '')
# Verbindung zur Datenbank herstellen
try:
    conn = mariadb.connect(
        user="cip_user",
        password="cip_pw",
        host="127.0.0.1",
        port=3306,
        database="CIP"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

conn.autocommit = False
cur = conn.cursor()

# Daten in die Datenbank einfügen
errors = []
for index, row in data.iterrows():
    query = """
    INSERT INTO tbl_financial_euronext_data
    (name, currency, market_cap_in_USD, carbon_footprint_total_GHG_emissions_per_enterprise_value, security_type, security_type_subtype, market, isin, industry, super_sector, sector, subsector, average_training_hours_per_employee, number_of_female_board_members, number_of_board_members, share_of_woman_in_total_workforce_in_percent, share_of_woman_in_management_bodies_in_percent, gender_pay_gap_in_percent, rate_of_employees_with_disabilities_in_percent, board_gender_diversity_in_percent, total_energy_consumption_in_mwh, rate_of_resignation_in_percent) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    try:
        cur.execute(query, (
            row['Name'],
            row['Currency'],
            row['Market Cap in USD'],
            row['Carbon footprint (total GHG emissions / enterprise value)_in_USD'],
            row['Type'],
            row['Sub type'],
            row['Market'],
            row['ISIN Code'],
            row['Industry'],
            row['SuperSector'],
            row['Sector'],
            row['Subsector'],
            row['Average training hours per employee'],
            row['Number of female board members'],
            row['Number of board members'],
            row['Share of women in total workforce in %'],
            row['Share of women in management bodies in %'],
            row['Gender pay gap in %'],
            row['Rate of employees with disabilities in %'],
            row['Board gender diversity (female board members / total board members) in %'],
            row['Total energy consumption in mwh'],
            row['Rate of resignation in %']
        ))
    except mariadb.Error as e:
        errors.append(f"Error: {e} at index {index}")
        continue

# Änderungen bestätigen und Verbindung schließen
conn.commit()


# Fehler und Erfolgsmeldung ausgeben
if errors:
    print(f"Es gab {len(errors)} Fehler beim Einfügen der Daten.")
for error in errors:
    print(error)
print("Data insertion complete.")

In [ ]:
try:
    cur.execute("SELECT name FROM tbl_financial_euronext_data")
    for name in cur:
        print(name)
except mariadb.Error as e:
    print(f"Error: {e}")

In [ ]:
conn.close()